In [122]:
import pypsa as psa
import networkx as nx
import numpy as np
import pandas as pd

In [123]:
def create_net():
    buses = ["Bus {}".format(i) for i in range(10)]
    snapshots = range(7)
    n = psa.Network()
    n.set_snapshots(snapshots)
    n.madd("Bus", buses)

    # add load as numpy array
    n.madd("Load",
           n.buses.index[:5] + " load",
           bus=buses[:5],
           p_set=[1,1,1,1,1])

    #use a suffix to avoid boilerplate to rename everything
    gen = ['Bus 7', 'Bus 8']
    for i in gen:
        n.add("Generator",
               name='my gen {}'.format(int(i[-1])),
               bus=i,
               p_nom_extendable=True,
               marginal_cost=float(int(i[-1]))/5)

    n.madd("Line", n.buses.index + ' lines',
           bus0=buses,
           bus1=np.roll(buses,1),
           r=0.01, x=0.01)
    return n

In [124]:
demand = pd.read_csv('demand.csv')
# demand.set_index('datetime', inplace=True)

In [125]:
n = create_net()
snapshots = pd.DatetimeIndex([])
for i in demand.datetime.to_list():
    period = pd.date_range(
            start=i,
            freq="{}H".format(1),
            periods=1)
    snapshots = snapshots.append(period)
print(snapshots)
# n.snapshots = pd.MultiIndex.from_arrays([snapshots.year, snapshots])
n.snapshots = snapshots
    # convert to multiindex and assign to network


DatetimeIndex(['2025-01-01 00:00:00', '2025-01-01 01:00:00',
               '2025-01-01 02:00:00', '2025-01-01 03:00:00',
               '2025-01-01 04:00:00', '2025-01-01 05:00:00',
               '2025-01-01 06:00:00', '2025-01-01 07:00:00',
               '2025-01-01 08:00:00', '2025-01-01 09:00:00',
               '2025-01-01 10:00:00', '2025-01-01 11:00:00',
               '2025-01-01 12:00:00', '2025-01-01 13:00:00',
               '2025-01-01 14:00:00', '2025-01-01 15:00:00',
               '2025-01-01 16:00:00', '2025-01-01 17:00:00',
               '2025-01-01 18:00:00', '2025-01-01 19:00:00',
               '2025-01-01 20:00:00', '2025-01-01 21:00:00',
               '2025-01-01 22:00:00', '2025-01-01 23:00:00'],
              dtype='datetime64[ns]', freq='H')


In [126]:
print(n.loads.index)
print(n.generators.index)

Index(['Bus 0 load', 'Bus 1 load', 'Bus 2 load', 'Bus 3 load', 'Bus 4 load'], dtype='object', name='Load')
Index(['my gen 7', 'my gen 8'], dtype='object', name='Generator')


In [127]:
# for i in n.loads.index.to_list():
temp_col = np.array(demand.type_A.to_list(), dtype=np.float16)
# n.add("Load", "Added", bus ="Bus 9",p_set=temp_col)

# n.loads.loc[i,'p_set'] = np.array(demand.type_A.to_list(), dtype=np.float16)

In [128]:
# count = 0
# ind = demand.columns[1:]
# print(ind)
# for i in n.loads.index.to_list():
#     temp_col = np.array(demand[ind[count]].to_list(), dtype=np.float16)
#     n.loads_t.p_set.loc[i] =temp_col
#     count+=1
#     if count>len(ind):
#         count =0

In [134]:
# n.pf()
count = 0
ind = demand.columns[1:]
print(ind)

for i in n.loads.index.to_list():
    temp_col = np.array(demand[ind[count]].to_list(), dtype=np.float16)
    count+=1
    if count>len(ind):
        count =0
    n.loads_t.p_set[i] = temp_col
n.pf()
print(n.loads_t.p_set)

INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots DatetimeIndex(['2025-01-01 00:00:00', '2025-01-01 01:00:00',
               '2025-01-01 02:00:00', '2025-01-01 03:00:00',
               '2025-01-01 04:00:00', '2025-01-01 05:00:00',
               '2025-01-01 06:00:00', '2025-01-01 07:00:00',
               '2025-01-01 08:00:00', '2025-01-01 09:00:00',
               '2025-01-01 10:00:00', '2025-01-01 11:00:00',
               '2025-01-01 12:00:00', '2025-01-01 13:00:00',
               '2025-01-01 14:00:00', '2025-01-01 15:00:00',
               '2025-01-01 16:00:00', '2025-01-01 17:00:00',
               '2025-01-01 18:00:00', '2025-01-01 19:00:00',
               '2025-01-01 20:00:00', '2025-01-01 21:00:00',
               '2025-01-01 22:00:00', '2025-01-01 23:00:00'],
              dtype='datetime64[ns]', name='snapshot', freq='H')
INFO:pypsa.pf:Newton-Raphson solved in 3 iterations with error of 0.000000 in 0.016397 seconds
INFO:pypsa.pf:Ne

Index(['type_A', 'type_B', 'type_C', 'type_D', 'type_E', 'type_F', 'type_G',
       'type_H', 'type_I', 'type_J', 'type_K'],
      dtype='object')


INFO:pypsa.pf:Newton-Raphson solved in 3 iterations with error of 0.000000 in 0.031302 seconds
INFO:pypsa.pf:Newton-Raphson solved in 3 iterations with error of 0.000000 in 0.025888 seconds
INFO:pypsa.pf:Newton-Raphson solved in 3 iterations with error of 0.000000 in 0.016894 seconds
INFO:pypsa.pf:Newton-Raphson solved in 3 iterations with error of 0.000000 in 0.025829 seconds
INFO:pypsa.pf:Newton-Raphson solved in 3 iterations with error of 0.000000 in 0.015671 seconds
INFO:pypsa.pf:Newton-Raphson solved in 3 iterations with error of 0.000000 in 0.025179 seconds
INFO:pypsa.pf:Newton-Raphson solved in 3 iterations with error of 0.000001 in 0.015677 seconds
INFO:pypsa.pf:Newton-Raphson solved in 3 iterations with error of 0.000000 in 0.031249 seconds
INFO:pypsa.pf:Newton-Raphson solved in 3 iterations with error of 0.000000 in 0.025708 seconds
INFO:pypsa.pf:Newton-Raphson solved in 3 iterations with error of 0.000000 in 0.015659 seconds
INFO:pypsa.pf:Newton-Raphson solved in 3 iteration

Load                 Bus 0 load  Bus 1 load  Bus 2 load  Bus 3 load  \
snapshot                                                              
2025-01-01 00:00:00         0.0    0.455078    0.596680    0.985840   
2025-01-01 01:00:00         1.0    0.544922    0.045288    0.980957   
2025-01-01 02:00:00         1.0    0.936523    0.166016    0.475586   
2025-01-01 03:00:00         0.0    0.388184    0.519043    0.393066   
2025-01-01 04:00:00         0.0    0.448975    0.545898    0.925293   
2025-01-01 05:00:00         0.0    0.079956    0.700684    0.550781   
2025-01-01 06:00:00         1.0    0.625000    0.619629    0.151001   
2025-01-01 07:00:00         0.0    0.573730    0.713379    0.352539   
2025-01-01 08:00:00         1.0    0.329346    0.275635    0.715820   
2025-01-01 09:00:00         1.0    0.848633    0.870117    0.982422   
2025-01-01 10:00:00         1.0    0.242188    0.879883    0.392090   
2025-01-01 11:00:00         0.0    0.080017    0.799805    0.868652   
2025-0

In [62]:
print(n.loads.p_set)

Load
Bus 0 load    0.0
Bus 1 load    0.0
Bus 2 load    0.0
Bus 3 load    0.0
Bus 4 load    0.0
Added         0.0
Name: p_set, dtype: float64


In [63]:
n.pf()

INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots DatetimeIndex(['2025-01-01 00:00:00', '2025-01-01 01:00:00',
               '2025-01-01 02:00:00', '2025-01-01 03:00:00',
               '2025-01-01 04:00:00', '2025-01-01 05:00:00',
               '2025-01-01 06:00:00', '2025-01-01 07:00:00',
               '2025-01-01 08:00:00', '2025-01-01 09:00:00',
               '2025-01-01 10:00:00', '2025-01-01 11:00:00',
               '2025-01-01 12:00:00', '2025-01-01 13:00:00',
               '2025-01-01 14:00:00', '2025-01-01 15:00:00',
               '2025-01-01 16:00:00', '2025-01-01 17:00:00',
               '2025-01-01 18:00:00', '2025-01-01 19:00:00',
               '2025-01-01 20:00:00', '2025-01-01 21:00:00',
               '2025-01-01 22:00:00', '2025-01-01 23:00:00'],
              dtype='datetime64[ns]', name='snapshot', freq='H')
INFO:pypsa.pf:Newton-Raphson solved in 0 iterations with error of 0.000000 in 0.000000 seconds
INFO:pypsa.pf:Ne

{'n_iter': SubNetwork           0
 snapshot              
 2025-01-01 00:00:00  0
 2025-01-01 01:00:00  3
 2025-01-01 02:00:00  3
 2025-01-01 03:00:00  0
 2025-01-01 04:00:00  0
 2025-01-01 05:00:00  0
 2025-01-01 06:00:00  3
 2025-01-01 07:00:00  0
 2025-01-01 08:00:00  3
 2025-01-01 09:00:00  3
 2025-01-01 10:00:00  3
 2025-01-01 11:00:00  0
 2025-01-01 12:00:00  0
 2025-01-01 13:00:00  3
 2025-01-01 14:00:00  3
 2025-01-01 15:00:00  0
 2025-01-01 16:00:00  0
 2025-01-01 17:00:00  3
 2025-01-01 18:00:00  3
 2025-01-01 19:00:00  3
 2025-01-01 20:00:00  3
 2025-01-01 21:00:00  3
 2025-01-01 22:00:00  3
 2025-01-01 23:00:00  0,
 'error': SubNetwork                      0
 snapshot                         
 2025-01-01 00:00:00  0.000000e+00
 2025-01-01 01:00:00  4.839795e-12
 2025-01-01 02:00:00  4.839795e-12
 2025-01-01 03:00:00  0.000000e+00
 2025-01-01 04:00:00  0.000000e+00
 2025-01-01 05:00:00  0.000000e+00
 2025-01-01 06:00:00  4.839795e-12
 2025-01-01 07:00:00  0.000000e+00
 2025-

In [68]:
print(n.buses.loc["Bus 9",:])

v_nom            1.0
type                
x                0.0
y                0.0
carrier           AC
unit            None
v_mag_pu_set     1.0
v_mag_pu_min     0.0
v_mag_pu_max     inf
control           PQ
sub_network        0
generator        NaN
Name: Bus 9, dtype: object
